<a href="https://github.com/amjadraza/ai-agents-collection/blob/main/Crewai/crewai_market_researcher_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Market Researcher using Crewai

## Goal
In this project, we are building a multi-agent system using the crewai library to perform market research on emerging trends in the restaurant industry. The focus areas are digital menus, AI in restaurants, and pricing strategies. The system consists of two main agents:

* Trend Research Agent: Gathers relevant data on specified topics.
* Market Researcher Agent: Synthesizes the gathered information into a coherent report.

## Setup
First, we install the necessary libraries and retrieve the API keys required for accessing OpenAI's language models and performing web searches.

In [28]:
%pip install -qU crewai crewai[tools]  langchain-community openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 11.2 MB/s eta 0:00:00


In [29]:
# Retrieve the API keys securely from Google Colab's user data
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')  # Openai API key for language models
os.environ['SERPER_API_KEY'] = userdata.get('SERPER_API_KEY')    # Serper API key for web search capabilities

Next, we import the essential libraries for agent creation and set the API keys as environment variables.

In [30]:
# Import essential libraries for agent creation and web tools
from crewai import Agent, Task, Crew, Process          # Core components for creating agents and tasks
from crewai_tools import SerperDevTool, ScrapeWebsiteTool  # Tools for web searching and scraping
from IPython.display import display, Markdown          # For displaying outputs in a Jupyter notebook
from langchain_openai import ChatOpenAI

## Tools
We define the tools that our agents will use to perform web searches and scrape website content.

In [31]:
# Initialize the tool for performing web searches using Serper API
search_tool = SerperDevTool()

# Initialize the tool for scraping website content
scrape_tool = ScrapeWebsiteTool()

The SerperDevTool allows agents to perform web searches, while the ScrapeWebsiteTool enables them to extract content from websites. These tools expand the agents' capabilities beyond text generation to include data retrieval from the internet.

## Research AI Agent + Task
We define the Trend Research Agent responsible for gathering relevant data.

In [33]:
# Define the Research AI Agent with its role, goal, and backstory
research_agent = Agent(
    role="Trend Research Agent",
    goal="Retrieve relevant data on digital menus, AI in restaurants, and pricing strategies",
    backstory="You are responsible for gathering relevant information on key trends in the restaurant industry for 2024 and beyond",
    llm=ChatOpenAI(model_name="gpt-4o-mini", temperature=0.8),  # Using GPT-4o-mini model
    tools=[search_tool, scrape_tool],  # Assigning the search and scrape tools
    verbose=True
)

The agent's role, goal, and backstory provide context that guides its behavior. By specifying the language model and tools, we define how the agent will perform its tasks. The temperature parameter in the language model influences the creativity of the output.

Tasks are assigned to agents, meaning that one agent can have multiple tasks.

In the crewai framework, agents can handle multiple tasks. This allows for efficient use of resources and enables agents to perform related tasks without the need for additional agents.

We create two tasks for the Trend Research Agent:

1. Trend Task: Research emerging trends in digital menus.
2. Pricing Task: Research new pricing strategies for restaurants.

In [34]:
# Create a task for researching trends in digital menus
trend_task = Task(
    description="Research emerging trends in 2024 and 2025 on digital menus",
    expected_output="All the information organized with key trends and insights for Bitte's MVP and future plans",
    agent=research_agent  # Assigning the task to the research agent
)

In [35]:
# Create a task for researching pricing strategies
pricing_task = Task(
    description="Research new pricing strategies for restaurants for takeaway and delivery",
    expected_output="All the retrieved information organized with key trends and insights for Bitte's MVP and future plans",
    agent=research_agent  # Assigning the task to the same agent
)

By assigning both tasks to the research_agent, we leverage its capabilities to gather all necessary data. The expected_output field defines what we anticipate receiving from the agent after task completion.

## Writer Agent
We define the Market Researcher Agent responsible for synthesizing the data into a coherent report.

In [36]:
# Define the writer agent to generate reports
writer_agent = Agent(
    role="Market Researcher that provides reports",
    goal="Generate reports based on search results",
    backstory="You are an experienced Market Researcher that synthesizes search results into a coherent report",
    llm=ChatOpenAI(model_name="gpt-4o-mini", temperature=0.8),  # Using GPT-40-mini model
    verbose=True
)

The writer_agent is designed to process the data collected by the research_agent and compile it into a structured report. The agent's role and backstory help guide its behavior during the task.

We define the writing task for the writer_agent, using the outputs of the previous tasks as context.


In [37]:
# Define the writing task
writing_task = Task(
    description="Generate a report based on the retrieved search data.",
    expected_output="A detailed and coherent report in Markdown on the trends in menu design, AI in restaurants, and pricing strategies. The report must start with an executive summary, followed by the content per topic, and conclude with actionable recommendations",
    agent=writer_agent,
    context=[trend_task, pricing_task]  # Using previous tasks as context
)

The context parameter allows the writer_agent to access the outputs from the trend_task and pricing_task. This is essential for the agent to incorporate the research findings into the final report.

## Assemble the team of AI Agents

We create a crew to manage the agents and tasks, specifying that tasks should be executed sequentially.

In [38]:
# Create the AI team for market research
crew = Crew(
    agents=[research_agent, writer_agent],  # List of agents involved
    tasks=[trend_task, pricing_task, writing_task],  # List of tasks to execute
    verbose=True,
    process=Process.sequential  # Ensuring tasks are processed in order
)
result = crew.kickoff()  # Start the task execution

# Agent: Trend Research Agent
## Task: Research emerging trends in 2024 and 2025 on digital menus


# Agent: Trend Research Agent
## Thought: I need to gather information on emerging trends related to digital menus in the restaurant industry for 2024 and 2025. This will involve searching for relevant data and insights.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"emerging trends digital menus restaurant industry 2024 2025\"}"
## Tool Output: 
{'searchParameters': {'q': 'emerging trends digital menus restaurant industry 2024 2025', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'Future Trends in Digital Menu Boards for 2024-2025 - DotSignage', 'link': 'https://www.dotsignage.com/blog/future-trends-in-digital-menu-boards/', 'snippet': 'Future trends of Digital Menu Boards in 2024 – Real time dynamic menus, AI integrations and gamification of digital menu boards.', 'position': 1}, {'title': 'The Era of Digital Menus in the Restaurant In

The Crew object orchestrates the agents and tasks. By setting process=Process.sequential, we ensure that tasks are completed in order, which is important when tasks depend on the outputs of previous tasks.

**ReACT prompt engineering technique**

ReACT (Reasoning and Acting) is a prompt engineering technique where the AI agent is designed to reason through tasks and decide on actions. It involves generating reasoning traces and actions that guide the agent towards the desired outcome. This technique improves the agent's ability to handle complex tasks by combining reasoning with action.

Finally, we extract and display the outputs from each task in a readable format using Markdown. We iterate over the task outputs and use Markdown to format the display. This helps in presenting the results in a structured and readable manner within a Jupyter notebook.


In [39]:
# Extract and display each agent's output in Markdown
for idx, task_output in enumerate(result.tasks_output):
  display(Markdown(f"### agent {idx +1}: task_output.agent]\n{task_output.raw}"))

### agent 1: task_output.agent]
### Key Trends and Insights on Digital Menus for 2024 and Beyond

#### 1. **AI and Personalization**
   - **Trend**: The integration of Artificial Intelligence (AI) in digital menus is expected to rise significantly. AI can analyze customer data and recommend dishes based on personal preferences and dietary restrictions, enhancing the dining experience.
   - **Insight**: Restaurants using AI-driven solutions for digital menus can expect increased customer engagement and satisfaction, leading to higher sales, particularly in recommendatory upselling of meals.

#### 2. **Dynamic and Real-time Updates**
   - **Trend**: Digital menus offer the ability to update items and prices in real-time, an essential feature for managing food costs and availability.
   - **Insight**: This flexibility allows restaurants to promote daily specials instantly and avoid customer disappointment from outdated information, thus improving overall customer satisfaction.

#### 3. **Interactive Nutritional Information**
   - **Trend**: The future of digital menus will include interactive elements where customers can access nutritional information about dishes instantly.
   - **Insight**: This caters to health-conscious diners and can enhance sales of healthier options when customers are provided with relevant information during the ordering process.

#### 4. **Gamification Elements**
   - **Trend**: Incorporating gamification into digital menus, such as loyalty programs or spin-to-win promotions, is gaining traction.
   - **Insight**: This interactive engagement can increase customer participation, encourage repeat visits, and enhance brand loyalty by making the dining experience more enjoyable.

#### 5. **Multi-Sensory Experiences**
   - **Trend**: Future digital menus may leverage multi-sensory experiences that include visuals, sounds, and even scents to create more immersive dining experiences.
   - **Insight**: Enhanced customer experiences through sensory engagement can lead to longer dining durations and higher spending per visit.

#### 6. **Automation of Ordering Processes**
   - **Trend**: Self-service kiosks and mobile ordering solutions are becoming more prevalent, allowing customers to place orders directly from their devices.
   - **Insight**: This reduces wait times and order errors, streamlining operations and freeing staff to provide enhanced service, which can lead to increased overall sales.

#### 7. **Augmented Reality (AR) Features**
   - **Trend**: The adoption of AR technology for enhanced visual representation of menu items is on the rise.
   - **Insight**: AR can offer customers a 3D view of their meal options, potentially increasing their willingness to try new items and enhancing their dining experience.

#### 8. **Data-Driven Decision Making**
   - **Trend**: Digital menus allow for extensive data collection on customer preferences and purchasing behaviors.
   - **Insight**: Restaurants can leverage this data to adjust menu offerings, optimize marketing strategies, and ultimately drive profitability.

#### 9. **Increased Efficiency and Cost Reduction**
   - **Trend**: By adopting digital menus, restaurants can decrease the need for manual updates and printing, thus saving on costs and reducing waste.
   - **Insight**: The operational efficiencies gained can enhance margins, especially in labor-intensive environments.

#### 10. **Enhanced Customer Engagement**
   - **Trend**: Digital menus facilitate better customer engagement through targeted promotions and personalized experiences.
   - **Insight**: Engaging customers effectively at every touchpoint can lead to improved retention rates and higher average transaction values.

### Conclusion
The integration of digital menus with AI, real-time updates, interactive features, and other advanced technologies is set to redefine dining experiences in 2024 and beyond. As restaurants increasingly focus on delivering personalized, engaging, and efficient service, these trends present a compelling opportunity for Bitte to innovate and enhance its offerings in the competitive restaurant landscape.

### agent 2: task_output.agent]
### Key Trends and Insights on Pricing Strategies for Restaurants in 2024 and Beyond

#### 1. **Dynamic Pricing**
   - **Trend**: Dynamic pricing is emerging as a significant strategy where restaurants adjust their prices in real-time based on demand and other factors such as time of day and delivery distance.
   - **Insight**: This approach allows restaurants to optimize revenue by lowering prices during off-peak hours and increasing them during high-demand periods, thereby maximizing profits while maintaining customer satisfaction.

#### 2. **Surge Pricing**
   - **Trend**: Inspired by the ride-sharing industry, surge pricing adjusts delivery fees based on real-time demand levels.
   - **Insight**: Implementing surge pricing offers opportunities for restaurants to capitalize on peak order times, encouraging customers to order during lower-demand periods by providing incentives like discounts.

#### 3. **Flexible Delivery Fees**
   - **Trend**: Many restaurants are adopting flexible delivery fees based on order size or distance.
   - **Insight**: By offering free delivery for larger orders or charging a reduced fee for nearby deliveries, restaurants can encourage larger purchases while still covering operational costs.

#### 4. **Value-Based Pricing**
   - **Trend**: Restaurants are moving towards value-based pricing strategies that focus on perceived value rather than just cost-plus pricing.
   - **Insight**: This method involves understanding customer preferences and willingness to pay, which can lead to higher profitability, particularly when premium items or experiences are involved.

#### 5. **Bundle Offers and Meal Kits**
   - **Trend**: Offering bundled meal deals or meal kits has become a popular pricing strategy.
   - **Insight**: This strategy not only provides convenience but also encourages higher spending per order, as customers see the value in purchasing a complete meal rather than individual items.

#### 6. **Subscription Models**
   - **Trend**: Some restaurants are exploring subscription models where customers pay a monthly fee for a set number of meals or discounts.
   - **Insight**: This creates a steady revenue stream for restaurants while promoting customer loyalty and repeat business.

#### 7. **Cost Transparency**
   - **Trend**: Increased transparency about pricing and cost structures is being requested by consumers.
   - **Insight**: Restaurants that communicate the rationale behind their pricing—such as ingredient quality or preparation methods—can enhance customer trust and justify higher prices.

#### 8. **Loyalty Programs with Tiered Pricing**
   - **Trend**: Implementing loyalty programs that offer tiered pricing based on customer spending levels.
   - **Insight**: This encourages repeat business and larger orders, as customers want to take advantage of better pricing options available to loyal patrons.

#### 9. **Utilizing Data Analytics**
   - **Trend**: Restaurants are increasingly employing data analytics to monitor customer behavior and adjust pricing strategies accordingly.
   - **Insight**: By leveraging data, restaurants can identify peak ordering times and customer preferences, allowing for more informed pricing decisions that align with market demand.

#### 10. **Focus on Delivery Service Partnerships**
   - **Trend**: Collaborating with delivery service platforms to optimize pricing strategies that benefit both parties.
   - **Insight**: By aligning pricing structures with delivery partners, restaurants can enhance their visibility and competitiveness in the food delivery market.

### Conclusion
As the restaurant industry navigates the evolving landscape of takeaway and delivery services, embracing innovative pricing strategies will be crucial. By leveraging dynamic pricing, surge pricing, value-based strategies, and data analytics, restaurants can enhance profitability while meeting customer expectations. These trends present significant opportunities for Bitte to develop effective pricing models that resonate with its target market and drive sustainable growth.

### agent 3: task_output.agent]
# Trends in Menu Design, AI in Restaurants, and Pricing Strategies

## Executive Summary
The restaurant industry is undergoing significant transformation through advancements in digital menus, the application of artificial intelligence (AI), and innovative pricing strategies. This report synthesizes current trends and insights that will shape dining experiences and operational efficiencies in 2024 and beyond. The integration of technology, particularly AI in menu design and pricing strategies, is driving increased customer engagement, personalized experiences, and optimized revenue management. This report concludes with actionable recommendations for restaurants to capitalize on these trends.

---

## Menu Design Trends

### 1. AI and Personalization
The integration of AI into digital menus is on the rise, enabling the analysis of customer data to recommend dishes based on individual preferences and dietary restrictions. This personalization enhances the dining experience, potentially leading to increased customer satisfaction and higher sales through upselling.

### 2. Dynamic and Real-time Updates
Digital menus facilitate real-time updates, allowing restaurants to manage food costs and availability efficiently. This capability enables the promotion of daily specials and helps avoid customer disappointment, significantly improving overall satisfaction.

### 3. Interactive Nutritional Information
Future digital menus will feature interactive nutritional information, catering to health-conscious diners. By providing relevant details during the ordering process, restaurants can boost sales of healthier options.

### 4. Gamification Elements
Incorporating gamification, such as loyalty programs or spin-to-win promotions, is gaining traction in digital menus. This interactive engagement can increase repeat visits and enhance brand loyalty by making the dining experience more enjoyable.

### 5. Multi-Sensory Experiences
Future digital menus may leverage multi-sensory experiences, incorporating visuals, sounds, and even scents. These enhancements can lead to longer dining durations and increased spending per visit.

### 6. Automation of Ordering Processes
The prevalence of self-service kiosks and mobile ordering solutions is reducing wait times and order errors, streamlining operations, and allowing staff to provide enhanced service.

### 7. Augmented Reality (AR) Features
AR technology is being adopted for enhanced visual representation of menu items. This feature provides customers with a 3D view of meal options, increasing their willingness to try new items.

### 8. Data-Driven Decision Making
Digital menus facilitate extensive data collection on customer preferences. Restaurants can leverage this data to optimize menu offerings and marketing strategies, driving profitability.

### 9. Increased Efficiency and Cost Reduction
Digital menus reduce the need for manual updates and printing, leading to cost savings and reduced waste, particularly in labor-intensive environments.

### 10. Enhanced Customer Engagement
Digital menus enable better customer engagement through targeted promotions and personalized experiences, leading to improved retention rates and higher average transaction values.

---

## Pricing Strategies 

### 1. Dynamic Pricing
Dynamic pricing allows restaurants to adjust prices in real time based on demand, time of day, and delivery distance. This strategy helps optimize revenue and maintain customer satisfaction by offering lower prices during off-peak hours.

### 2. Surge Pricing
Inspired by the ride-sharing industry, surge pricing adjusts delivery fees based on real-time demand. This model can encourage customers to order during lower-demand periods by providing incentives like discounts.

### 3. Flexible Delivery Fees
Adopting flexible delivery fees based on order size or distance encourages larger purchases while covering operational costs. Free delivery for larger orders can be an effective strategy.

### 4. Value-Based Pricing
Restaurants are increasingly adopting value-based pricing, focusing on the perceived value of menu items rather than cost-plus pricing. Understanding customer preferences can lead to higher profitability.

### 5. Bundle Offers and Meal Kits
Offering bundled meal deals or meal kits provides convenience and encourages higher spending per order as customers see value in purchasing a complete meal.

### 6. Subscription Models
Some restaurants are exploring subscription models where customers pay a monthly fee for meals or discounts, creating a steady revenue stream and promoting customer loyalty.

### 7. Cost Transparency
Communicating the rationale behind pricing—such as ingredient quality—enhances customer trust and justifies higher prices.

### 8. Loyalty Programs with Tiered Pricing
Implementing loyalty programs with tiered pricing encourages repeat business, as customers seek to benefit from better pricing options available to loyal patrons.

### 9. Utilizing Data Analytics
Data analytics allows restaurants to monitor customer behavior and adjust pricing strategies effectively, aligning with market demand.

### 10. Focus on Delivery Service Partnerships
Collaborating with delivery service platforms to optimize pricing strategies can enhance visibility and competitiveness in the food delivery market.

---

## Conclusion
The restaurant industry is poised for a transformation driven by digital menus, AI integration, and innovative pricing strategies. By embracing these trends, restaurants can enhance customer experiences, improve operational efficiencies, and drive profitability. For Bitte, these insights present significant opportunities to innovate and develop effective strategies that resonate with their target market.

## Actionable Recommendations
1. Implement AI-driven digital menus for personalized customer experiences.
2. Utilize real-time data analytics to optimize menu offerings and pricing strategies.
3. Explore gamification and loyalty programs to enhance customer engagement.
4. Adopt flexible pricing models to maximize revenue during peak and off-peak times.
5. Develop transparent communication strategies regarding pricing to build customer trust.

By following these recommendations, restaurants can stay competitive and thrive in the evolving landscape of the dining industry.